## RQ3 Single-Turn — Suspicious-only Candidate Prep (No Penalty)

This notebook:
1. Loads a JSON dataset of examples
2. Marks examples as **suspicious** using rule-based checks
3. Writes two files:
- `suspicious_examples.json` (only suspicious examples)
- `all_with_suspicious_flag.json` (all examples + a boolean flag)

In [ ]:
import json

# TODO: set this to your input file path
INPUT_JSON = "data.json"

with open(INPUT_JSON, "r", encoding="utf-8") as f:
    examples = json.load(f)

print("Total examples:", len(examples))
print("First example keys:", list(examples[0].keys()) if examples else "N/A")

### Helper: Extract the 4 rubric scores



In [1]:
def extract_scores(ex):
    return {
        "accuracy": ex["accuracy"]["score"],
        "completeness": ex["completeness"]["score"],
        "relevance": ex["relevance"]["score"],
        "clarity": ex["clarity"]["score"],
    }

### Suspicious-only intervention rule


In [ ]:
def is_suspicious(scores):
    a, c, r, l = scores["accuracy"], scores["completeness"], scores["relevance"], scores["clarity"]

    return (
        (a == 1 and l == 3) or
        (a >= 2 and r == 1) or
        (a == 2 and r == 2 and l == 2) or
        (a + r + l <= 5) or
        (abs(a - r) >= 2) or
        (a == 3 and (r <= 1 or l <= 1))
    )

## Filter and save suspicious examples

This writes `suspicious_examples.json` containing only examples that are suspicious.

It also writes `all_with_suspicious_flag.json` containing every example with `intervention_required` set.

In [ ]:
import json

SUSPICIOUS_JSON = "suspicious_examples.json"
ALL_FLAGGED_JSON = "all_with_suspicious_flag.json"

suspicious_examples = []
all_flagged = []

for ex in examples:
    scores = extract_scores(ex)
    flag = bool(is_suspicious(scores))

    ex2 = dict(ex)
    ex2["scores_flat"] = scores
    ex2["intervention_required"] = flag
    all_flagged.append(ex2)

    if flag:
        suspicious_examples.append(ex2)

print("Suspicious examples:", len(suspicious_examples))

with open(SUSPICIOUS_JSON, "w", encoding="utf-8") as f:
    json.dump(suspicious_examples, f, indent=2, ensure_ascii=False)

with open(ALL_FLAGGED_JSON, "w", encoding="utf-8") as f:
    json.dump(all_flagged, f, indent=2, ensure_ascii=False)

print("Wrote:", SUSPICIOUS_JSON)
print("Wrote:", ALL_FLAGGED_JSON)


Shows a few suspicious example IDs and their flat scores.

In [ ]:
for ex in suspicious_examples[:10]:
    print(ex.get("id"), ex.get("scores_flat"))